In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import category_encoders as ce

In [2]:
labels = pd.read_csv("data/Labels.csv")
train_values = pd.read_csv("data/Training_values.csv")
test_values = pd.read_csv("data/Test_values.csv")
all_dataset = pd.read_csv("data/merged_dataset.csv")
training_NEW= pd.read_csv("data/training_NEW.csv")
training_labels=pd.read_csv("data/trainingLabels_NEW.csv")

In [3]:
# We'll be using the sample training dataset, and merge with the labels dataset
training_NEW = labels.merge(train_values, left_on='id', right_on='id')
training_NEW.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
print(training_NEW.shape[0])
training_NEW.shape[1]

59400


41

In [5]:
training_NEW.columns

Index(['id', 'status_group', 'amount_tsh', 'date_recorded', 'funder',
       'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name',
       'num_private', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'recorded_by', 'scheme_management', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [6]:
pd.set_option('display.max_columns', None)
training_NEW.dtypes

id                         int64
status_group              object
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment   

In [7]:
# This function conveniently calculates the number and percentage of missing values for each column in a given DataFrame.
def calculate_missing_data(df):
    # Calculate the total number of missing values in each column
    total = df.isnull().sum().sort_values(ascending=False)

    # Calculate the percentage of missing values in each column
    percent_1 = df.isnull().sum() / df.isnull().count() * 100

    # Round the percentage values to one decimal place and sort ascending values, for fully optimized readability!
    percent_2 = (round(percent_1, 1)).sort_values(ascending=False)

    # Create a dataframe to show the values, featuring the results along the columns axis
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data
calculate_missing_data(training_NEW)

,Total,%
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6
basin,0,0.0
lga,0,0.0
district_code,0,0.0


In [8]:
# Define a function to return any columns with only 1 unique value
def find_single_value_columns(df):
     
    # Create an empty list to store the names
    single_value_columns = []
    # Loop through each column
    for column in df.columns:
        # Using .unique(), if the length of the unique values list equals 1, it means the column has only a single unique value
        if len(df[column].unique()) == 1:
            single_value_columns.append(column)
    return single_value_columns

result = find_single_value_columns(training_NEW)
print(result)

['recorded_by']


In [9]:
# Drop the noisy column
training_NEW = training_NEW.drop(['recorded_by'], axis=1)

In [10]:
# There are a few good methods to accurately predict 'construction_year', but for now we'll replace '0' values with the median '1993'
training_NEW['construction_year'] = training_NEW['construction_year'].replace({0:1993})

# For other columns missing values, we'll replace with 'No Record'
# They'll be dealt with later with the function `reverse_cardinality_check`
training_NEW.scheme_name= training_NEW.scheme_name.fillna('No Record')
training_NEW.scheme_management = training_NEW.scheme_management.fillna('No Record')
training_NEW.installer = training_NEW.installer.fillna('No Record')
training_NEW.funder = training_NEW.funder.fillna('No Record')
training_NEW.public_meeting = training_NEW.public_meeting.fillna('No Record')
training_NEW.permit = training_NEW.permit.fillna('No Record')
training_NEW.subvillage = training_NEW.subvillage.fillna('No Record')

In [11]:
calculate_missing_data(training_NEW).head(10)

,Total,%
waterpoint_type_group,0,0.0
waterpoint_type,0,0.0
ward,0,0.0
lga,0,0.0
district_code,0,0.0
region_code,0,0.0
region,0,0.0
subvillage,0,0.0
basin,0,0.0
num_private,0,0.0


# Feature Engineering:
#### Engineering new numerical features will increase distinction between pumps, help to capture additional patterns, and overall benefit the training of ML algorithms.

In [12]:
# To engineer a feature for water pump 'age', we convert 'date_recorded' to string type and slice the first 4 characters to obtain the year (assuming 'YYYY-MM-DD' format), and convert this sliced string back to integer type
# Then we subtract 'construction_year' from the 'date_recorded' year to compute the age of the water pump, estimated from the data
training_NEW['age'] = training_NEW['date_recorded'].astype(str).str[:4].astype(int) - training_NEW['construction_year']

In [13]:
# To engineer a feature 'pop/year', reflecting the population use on each water pump, per year, since initial construction,
# First replace '0' values in 'population' and 'age' columns with '1' to avoid division by zero
# Then, divide 'population' by 'age' to obtain the average population served by the pump per year since its construction
training_NEW['pop/year'] = training_NEW['population'].replace({0:1}) / training_NEW['age'].replace({0:1})

In [14]:
# To engineer the feature `water_/_person`, first we replace '0' values with '1' to avoid division by zero
# Then we divide 'amount_tsh' by the 'population'
training_NEW['water_/_person'] = training_NEW['amount_tsh'].replace({0:1}) / training_NEW['population'].replace({0:1})

In [15]:
# Check
columns = training_NEW.columns
training_NEW[columns[-3:]]
# Now That's Amore!

,age,pop/year,water_/_person
0,12,9.083333,55.045872
1,3,93.333333,0.003571
2,4,62.500000,0.100000
3,27,2.148148,0.017241
4,18,0.055556,1.000000
...,...,...,...
59395,14,8.928571,0.080000
59396,15,3.733333,83.928571
59397,18,0.055556,1.000000
59398,18,0.055556,1.000000


# Encoding:

In [16]:
# For this projects purpose of binary classification, we'll only be observing 'functional' or 'broken' water pumps
# We'll replace the 'functional needs repair' category with 'functional'
training_NEW.loc[training_NEW['status_group'] == 'functional needs repair', 'status_group'] = 'functional'

In [17]:
label_encoder = preprocessing.LabelEncoder()

# Replace the 'status_group' column with encoded values
df_status=training_NEW[['id', 'status_group']]
df_status['status_group']= label_encoder.fit_transform(df_status['status_group'])  

<ipython-input-17-4bfcb5704470>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_status['status_group']= label_encoder.fit_transform(df_status['status_group'])


In [18]:
# This function will search the dataframe for features above the cardinality limit, then create a dict from the results

# High cardinality categorical variables can be problematic because they can result in a large number of input features after encoding, resulting in overfitting and noisy data
# This function could identify features that might cause issues when building models
def reverse_cardinality_check(n, training_NEW):
  # The 'n' parameter is a user-defined threshold. Any column with unique values greater than 'n'' will be considered high cardinality.
  
  # Create an empty list to store feature names with high cardinality
  feature_list = []
  
  # Create an empty list to store the number of unique values (cardinality)
  cardinality_value = []
  
  for _ in range(len(training_NEW.columns)):
    # If a column's `cardinality` is greater than 'n', append the column name and its cardinality to their respective lists
    if len(training_NEW[training_NEW.columns[_]].value_counts()) > n:
      
      feature_list.append(training_NEW.columns[_])
      
      cardinality_value.append(len(training_NEW[training_NEW.columns[_]].value_counts()))
  # Combine the two lists into a dictionary where each key-value pair represents a high-cardinality feature and its cardinality                                  
  feature_dict = dict(zip(feature_list, cardinality_value))
  
  return feature_dict


In [19]:
# Now, we will seperate low_card features for One-Hot Encoding, and high_card features for Ordinal Encoding

# This returns a dict. of features with cardinality greater than 150
high_card_feature_dict = reverse_cardinality_check(150, training_NEW)

# Refract columns with high_card from the dataframe using the keys from the dict.
high_card_features = training_NEW[list(high_card_feature_dict.keys())]

# The remaining columns are low_card, so we drop the high_card columns directly from `training_NEW`
low_card_features = training_NEW.drop(columns = list(high_card_feature_dict.keys()))

# Apply OHE to low_card
one_hot_encode = ce.OneHotEncoder(use_cat_names=True)
one_hot_encode.fit(low_card_features)
low_card_features = one_hot_encode.transform(low_card_features)

# Apply OE to high_card
ordinal_encode = ce.OrdinalEncoder()
ordinal_encode.fit(high_card_features)
high_card_features = ordinal_encode.transform(high_card_features)

# Combine low and high cardinality back into a 'features' dataframe

# This stores the results in a list
frames =[low_card_features, high_card_features]
# This line concatenates the list along the column axis
features = pd.concat(frames, axis = 1)

# Then, merge the target prediction values from `df_status` with the newly encoded 'features' dataframe based on the 'id'
df = df_status.merge(features, left_on='id', right_on='id')
df.to_csv('data/Prepared_DF.csv', index=False)